In [14]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange
import rosbags
import json
import open3d as o3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [12]:
file_path = 'datasets/testing.json'
# Open and read the JSON file

with open(file_path, 'r') as file:
    data = json.load(file)

# Now 'data' is a dictionary
num_key = len(data.keys())
if num_key > 0:
    print(f"Data loaded, number of keys = {num_key}")

Data loaded, number of keys = 47


In [19]:
frame = 40
points = np.array(data[str(frame)])

In [22]:
data[str(0)][0:10]

[[1.0529999732971191, 0.8410000205039978, -0.20000000298023224],
 [0.9900000095367432, 0.7870000004768372, -0.16699999570846558],
 [1.0679999589920044, 0.847000002861023, -0.15800000727176666],
 [1.065999984741211, 0.8420000076293945, -0.13600000739097595],
 [1.406999945640564, 1.1100000143051147, -0.15199999511241913],
 [1.4420000314712524, 1.1369999647140503, -0.12700000405311584],
 [1.0540000200271606, 0.8460000157356262, -0.20000000298023224],
 [0.9990000128746033, 0.7979999780654907, -0.1679999977350235],
 [1.024999976158142, 0.8159999847412109, -0.15199999511241913],
 [1.0609999895095825, 0.8429999947547913, -0.13500000536441803]]

In [23]:
data[str(40)][0:10]

[[1.503999948501587, 0.3569999933242798, -0.2709999978542328],
 [1.7829999923706055, 0.42100000381469727, -0.2919999957084656],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [2.0460000038146973, 0.4740000069141388, -0.20200000703334808],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [1.0199999809265137, 0.23100000619888306, -0.13300000131130219]]

In [20]:
# Convert numpy array to open3d point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])